In [1]:
%load_ext autoreload
%autoreload 2

from gpu import set_gpu
import numpy as np
import os
import tensorflow as tf
from adapt.ml.dl_subclass import MLP, MLPprop, CNN, CNNprop, get_train, get_test
import adapt.utils.data_utils as prd
from adapt.ml.lda import train_lda, eval_lda
import adapt.loop as lp

set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [2]:
path = 'C:/Users/yteh/Documents/work/necal/home data/TR58/DATA/MAT/'
all_files = os.listdir(path)
train_file = all_files[0]
adapt_file = all_files[1]
test_file = all_files[2]

In [3]:
train_data, train_params = prd.load_caps_train(path + train_file + '/traindata.mat')
train_data = train_data[:,:8,:]

train_dof = np.unique(train_params[:,2])
key = np.empty(train_dof.shape)
for i in range(len(train_dof)):
    key[i] = train_params[np.argmax(train_params[:,2] == train_dof[i]),0]

In [4]:
# Initial training
ep = 30
n_dof = len(train_dof)

trainmlp, traincnn, y_train, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, emg_scale, scaler, _, _, _ = prd.prep_train_caps(train_data, train_params, prop_b = False)
mlp, cnn, w, c = lp.train_models(traincnn, trainmlp, x_train_lda, y_train_lda, n_dof, ep)

Epoch 1,  Loss: 1.60,  Accuracy: 42.65 
Epoch 30,  Loss: 0.34,  Accuracy: 87.23 
Epoch 1,  Loss: 1.21,  Accuracy: 57.54 
Epoch 30,  Loss: 0.11,  Accuracy: 96.23 


In [5]:
# Test all controllers without recalibration or adaptation
acc = np.empty((len(all_files)-1,3))

for i in range(1,len(all_files)):
    # load data
    test_file = all_files[i]
    test_data, test_params = prd.load_caps_train(path + test_file + '/traindata.mat')
    test_data = test_data[:,:8,:].astype('float64')
    
    # check class labels
    test_data, test_params = lp.check_labels(test_data,test_params,train_dof,key)

    # test 
    y_test, x_test_mlp, x_test_cnn, x_lda, y_lda = prd.prep_test_caps(test_data, test_params, scaler, emg_scale, num_classes=len(train_dof))
    acc[i-1,:] = lp.test_models(x_test_cnn, x_test_mlp, x_lda, y_test, y_lda, cnn, mlp, w, c)

    print ('Set: ' + test_file, f'CNN Accuracy: {acc[i-1,0]:.2f},', f'MLP Accuracy: {acc[i-1,1]:.2f},', f'LDA Accuracy: {acc[i-1,2]:.2f}')

Set: 20170730_054913 CNN Accuracy: 68.18, MLP Accuracy: 72.69, LDA Accuracy: 62.54
Set: 20170730_055812 CNN Accuracy: 68.10, MLP Accuracy: 71.26, LDA Accuracy: 71.13
Set: 20170730_060537 CNN Accuracy: 64.52, MLP Accuracy: 66.88, LDA Accuracy: 66.16
Set: 20170730_123937 CNN Accuracy: 67.00, MLP Accuracy: 67.59, LDA Accuracy: 65.40
Set: 20170731_123147 CNN Accuracy: 63.72, MLP Accuracy: 68.98, LDA Accuracy: 52.47
Set: 20170801_053500 CNN Accuracy: 62.37, MLP Accuracy: 62.03, LDA Accuracy: 55.88
Set: 20170801_054938 CNN Accuracy: 58.95, MLP Accuracy: 63.67, LDA Accuracy: 51.92
Set: 20170801_065547 CNN Accuracy: 73.66, MLP Accuracy: 71.22, LDA Accuracy: 65.23
Set: 20170802_050945 CNN Accuracy: 64.90, MLP Accuracy: 66.96, LDA Accuracy: 55.58
Set: 20170802_051732 CNN Accuracy: 62.41, MLP Accuracy: 63.55, LDA Accuracy: 62.41
Set: 20170802_052140 CNN Accuracy: 63.46, MLP Accuracy: 63.51, LDA Accuracy: 56.60
Set: 20170807_153152 CNN Accuracy: 61.31, MLP Accuracy: 65.53, LDA Accuracy: 53.06
Set:

In [6]:
# Test all controllers with recalibration
ep = 30
# Remove extra file if odd number of files
if len(all_files)%2 == 0:
    all_files[-1] = []
recal_acc = np.empty((len(all_files)-1,3))
recal_acc[:] = np.nan

for i in range(1,len(all_files)-1,2):
    # load recalibration data
    recal_file = all_files[i]
    recal_data, recal_params = prd.load_caps_train(path + recal_file + '/traindata.mat')
    recal_data = recal_data[:,:8,:]

    recal_dof = np.unique(recal_params[:,2])
    recal_key = np.empty(recal_dof.shape)
    for dof_i in range(len(recal_dof)):
        recal_key[dof_i] = recal_params[np.argmax(recal_params[:,2] == recal_dof[dof_i]),0]
    n_dof = len(recal_dof)
    
    recalmlp, recalcnn, _, _, _, x_train_lda, y_train_lda, emg_scale, scaler, _, _, _ = prd.prep_train_caps(recal_data, recal_params, prop_b = False)
    mlp, cnn, w, c = lp.train_models(recalcnn, recalmlp, x_train_lda, y_train_lda, n_dof, ep)
    
    # load data
    test_file = all_files[i+1]
    test_data, test_params = prd.load_caps_train(path + test_file + '/traindata.mat')
    test_data = test_data[:,:8,:].astype('float64')
    
    # check class labels
    test_data, test_params = lp.check_labels(test_data,test_params,recal_dof,recal_key)

    # test 
    y_test, x_test_mlp, x_test_cnn, x_lda, y_lda = prd.prep_test_caps(test_data, test_params, scaler, emg_scale, num_classes=n_dof)
    recal_acc[i+1,:] = lp.test_models(x_test_cnn, x_test_mlp, x_lda, y_test, y_lda, cnn, mlp, w, c)

    print ('Recal: ' + recal_file + ', Test: ' + test_file, f', CNN Accuracy: {recal_acc[i+1,0]:.2f},', f'MLP Accuracy: {recal_acc[i+1,1]:.2f},', f'LDA Accuracy: {recal_acc[i+1,2]:.2f}')

Epoch 1,  Loss: 1.57,  Accuracy: 46.49 
Epoch 30,  Loss: 0.35,  Accuracy: 86.66 
Epoch 1,  Loss: 1.26,  Accuracy: 55.03 
Epoch 30,  Loss: 0.13,  Accuracy: 95.35 
Recal: 20170728_103239, Test: 20170730_054913 , CNN Accuracy: 68.18, MLP Accuracy: 72.69, LDA Accuracy: 62.54
Epoch 1,  Loss: 1.49,  Accuracy: 49.29 
Epoch 30,  Loss: 0.21,  Accuracy: 92.49 
Epoch 1,  Loss: 1.23,  Accuracy: 57.15 
Epoch 30,  Loss: 0.07,  Accuracy: 97.58 
Recal: 20170730_055812, Test: 20170730_060537 , CNN Accuracy: 64.52, MLP Accuracy: 66.88, LDA Accuracy: 66.16
Epoch 1,  Loss: 1.52,  Accuracy: 47.56 
Epoch 30,  Loss: 0.15,  Accuracy: 94.87 
Epoch 1,  Loss: 1.01,  Accuracy: 67.85 
Epoch 30,  Loss: 0.03,  Accuracy: 98.86 
Recal: 20170730_123937, Test: 20170731_123147 , CNN Accuracy: 63.72, MLP Accuracy: 68.98, LDA Accuracy: 52.47
Epoch 1,  Loss: 1.82,  Accuracy: 29.84 
Epoch 30,  Loss: 0.35,  Accuracy: 87.21 
Epoch 1,  Loss: 1.37,  Accuracy: 53.84 
Epoch 30,  Loss: 0.14,  Accuracy: 95.36 
Recal: 20170801_053500

In [ ]:
# Test all controllers with adaptation
ep = 5
# Remove extra file if odd number of files
if len(all_files)%2 == 0:
    all_files[-1] = []
adapt_acc = np.empty((len(all_files)-1,3))
adapt_acc[:] = np.nan

for i in range(1,len(all_files)-1,2):
    # load recalibration data
    adapt_file = all_files[i]
    adapt_data, adapt_params = prd.load_caps_train(path + adapt_file + '/traindata.mat')
    adapt_data = adapt_data[:,:8,:]

    # check class labels
    adapt_data, adapt_params = lp.check_labels(adapt_data, adapt_params, train_dof, key)
    
    adaptmlp, adaptcnn, _, _, _, x_train_lda, y_train_lda, emg_scale, scaler, _, _, _ = prd.prep_train_caps(adapt_data, adapt_params, prop_b = False)
    mlp, cnn, w, c = lp.train_models(adaptcnn, adaptmlp, x_train_lda, y_train_lda, n_dof, ep, mlp, cnn)
    
    # load data
    test_file = all_files[i+1]
    test_data, test_params = prd.load_caps_train(path + test_file + '/traindata.mat')
    test_data = test_data[:,:8,:].astype('float64')
    
    # check class labels
    test_data, test_params = lp.check_labels(test_data, test_params, train_dof, key)

    # test 
    y_test, x_test_mlp, x_test_cnn, x_lda, y_lda = prd.prep_test_caps(test_data, test_params, scaler, emg_scale, num_classes=n_dof)
    adapt_acc[i+1,:] = lp.test_models(x_test_cnn, x_test_mlp, x_lda, y_test, y_lda, cnn, mlp, w, c)

    print ('Recal: ' + adapt_file + ', Test: ' + test_file, f', CNN Accuracy: {adapt_acc[i+1,0]:.2f},', f'MLP Accuracy: {adapt_acc[i+1,1]:.2f},', f'LDA Accuracy: {adapt_acc[i+1,2]:.2f}')

In [ ]:
for i in range(1,len(all_files)):
    # load data
    test_file = all_files[i]
    test_data, test_params = prd.load_caps_train(path + test_file + '/traindata.mat')
    test_data = test_data[:,:8,:].astype('float64')
    
    # check class labels
    test_data, test_params = lp.check_labels(test_data,test_params,train_dof,key)

    # test 
    y_test, x_test_mlp, x_test_cnn, x_lda, y_lda = prd.prep_test_caps(test_data, test_params, scaler, emg_scale, num_classes=len(train_dof))
    acc[i-1,:] = lp.test_models(x_test_cnn, x_test_mlp, x_lda, y_test, y_lda, cnn, mlp, w, c)

    print ('Set: ' + test_file, f'CNN Accuracy: {acc[i-1,0]:.2f},', f'MLP Accuracy: {acc[i-1,1]:.2f},', f'LDA Accuracy: {acc[i-1,2]:.2f}')

In [7]:
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

y_test, x_test_mlp, x_test_cnn, x_lda, y_lda = prd.prep_test_caps(adapt_data, adapt_params, scaler, emg_scale)

# test CNN
test_mod = get_test()
test_mod(x_test_cnn, y_test, cnn, test_loss, test_accuracy)
print (f'CNN Accuracy: {test_accuracy.result()*100:.2f}')

# test MLP
test_loss.reset_states()
test_accuracy.reset_states()

test_mod = get_test()
test_mod(x_test_mlp, y_test, mlp, test_loss, test_accuracy)
print (f'MLP Accuracy: {test_accuracy.result()*100:.2f}')

# test LDA
acc = eval_lda(w, c, x_lda, y_lda)
print (f'LDA Accuracy: {acc*100:.2f}')

CNN Accuracy: 36.87
MLP Accuracy: 69.24
LDA Accuracy: 62.54


In [10]:
ep = 5

trainmlp, traincnn, y_train, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, emg_scale, scaler, x_min, x_max, prop = prd.prep_train_caps(adapt_data, adapt_params, prop_b = False)

# Train neural networks
models = [mlp, cnn]
for model in models:
    if isinstance(model,CNN):
        ds = traincnn
    else:
        ds = trainmlp
    
    prop_b = isinstance(model, MLPprop) or isinstance(model, CNN)
    train_mod = get_train(prop = prop_b)

    for epoch in range(ep):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()

        for x, y, y2 in ds:
            if prop_b:
                train_mod(x, y, model, optimizer, train_loss, train_accuracy, train_prop_accuracy, y2)
            else:
                train_mod(x, y, model, optimizer, train_loss, train_accuracy)

        if epoch == 0 or epoch == ep-1:
            print(
                f'Epoch {epoch + 1}, '
                f'Loss: {train_loss.result():.2f}, '
                f'Accuracy: {train_accuracy.result() * 100:.2f} '
            )
    del train_mod

# Train LDA
w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)

mlp_w = mlp.get_weights()
cnn_w = cnn.get_weights()

Epoch 1, Loss: 0.68, Accuracy: 76.47 
Epoch 5, Loss: 0.37, Accuracy: 85.70 
Epoch 1, Loss: 0.61, Accuracy: 81.65 
Epoch 5, Loss: 0.21, Accuracy: 92.64 


In [8]:
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')
test_loss.reset_states()
test_accuracy.reset_states()

test_data, test_params = prd.load_caps_train(path + test_file + '/traindata.mat')
test_data = test_data[:,:8,:].astype('float64')
y_test, x_test_mlp, x_test_cnn, x_lda, y_lda = prd.prep_test_caps(test_data, test_params, scaler, emg_scale)

# test CNN
test_mod = get_test()
test_mod(x_test_cnn, y_test, cnn, test_loss, test_accuracy)
print (f'CNN Accuracy: {test_accuracy.result()*100:.2f}')

# test MLP
del test_mod
test_loss.reset_states()
test_accuracy.reset_states()

test_mod = get_test()
test_mod(x_test_mlp, y_test, mlp, test_loss, test_accuracy)
print (f'MLP Accuracy: {test_accuracy.result()*100:.2f}')

# test LDA
acc = eval_lda(w, c, x_lda, y_lda)
print (f'LDA Accuracy: {acc*100:.2f}')

CNN Accuracy: 36.09
MLP Accuracy: 67.80
LDA Accuracy: 71.13
